In [ ]:
!pip install tensorflow

In [138]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import random
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler as sc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.pipeline import Pipeline  
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks

In [ ]:
df = pd.read_csv("https://minio.lab.sspcloud.fr/jbrablx/ai_insurance/raw/train.csv")
df.head()

In [118]:
#df.drop(['id'], axis=1, inplace=True)
num_features = ['Age', 'Annual_Premium', 'Vintage']  
cat_features = ['Gender', 'Vehicle_Age', 'Driving_License', 'Region_Code', 'Previously_Insured', 'Vehicle_Damage', 'Policy_Sales_Channel'] 
df.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [119]:
age=pd.cut(df['Age'],bins=5,labels=['A1','A2','A3','A4','A5'])
rc=pd.cut(df['Region_Code'],bins=5,labels=['A1','A2','A3','A4','A5'])
ap=pd.cut(df['Annual_Premium'],bins=5,labels=['A1','A2','A3','A4','A5'])
vin=pd.cut(df['Vintage'],bins=5,labels=['A1','A2','A3','A4','A5'])

In [120]:
df['age']=age             
df['rc']=rc 
df['ap']=ap
df['vin']=vin

In [121]:
df_dum=pd.get_dummies(df[['age','rc','ap','vin','Gender','Vehicle_Age','Vehicle_Damage','Vehicle_Damage']])

In [122]:
df=pd.concat([df,df_dum],axis=1)

In [123]:
df.drop(['Gender','Age','age','Region_Code','rc','Vehicle_Age','Vehicle_Damage','Annual_Premium','ap','Vintage','vin'],axis=1,inplace=True)

In [124]:
X_train, X_test, y_train, y_test = train_test_split(df, df['Response'], test_size=0.1, random_state=42)

In [126]:
scaler=sc()
scaled_df_train=scaler.fit_transform(X_train.drop(['id','Response'],axis=1))
scaled_df_test=scaler.fit_transform(X_test.drop(['id','Response'],axis=1))

In [127]:
X_train, X_val, y_train, y_val = train_test_split(scaled_df_train, y_train, test_size=0.15, random_state=42)

In [128]:
sgc=SGDClassifier(penalty='l1',loss='modified_huber',early_stopping=True)
model_params={'alpha':[0.0012,0.0011,0.0013],
              'learning_rate':['invscaling'],
              'max_iter':[690,700,710],
              'validation_fraction':[0.46,0.47,0.48],
              'eta0':[0.65,0.66,0.64]}
ran=RandomizedSearchCV(sgc,param_distributions=model_params,cv=5,n_jobs=-1,verbose=2,n_iter=100)

In [129]:
ran.fit(scaled_df,df['Response'])

Fitting 5 folds for each of 81 candidates, totalling 405 fits


/opt/mamba/lib/python3.11/site-packages/sklearn/model_selection/_search.py:318: UserWarning: The total space of parameters 81 is smaller than n_iter=100. Running 81 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=SGDClassifier(early_stopping=True,
                                           loss='modified_huber',
                                           penalty='l1'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'alpha': [0.0012, 0.0011, 0.0013],
                                        'eta0': [0.65, 0.66, 0.64],
                                        'learning_rate': ['invscaling'],
                                        'max_iter': [690, 700, 710],
                                        'validation_fraction': [0.46, 0.47,
                                                                0.48]},
                   verbose=2)

In [130]:
ran.best_params_

{'validation_fraction': 0.46,
 'max_iter': 710,
 'learning_rate': 'invscaling',
 'eta0': 0.64,
 'alpha': 0.0013}

In [ ]:
ran.best_score_

In [132]:
y_pred=ran.predict(scaled_df_test)

In [ ]:
submissions=pd.DataFrame({'id':X_test['id'],'Response':y_pred})

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels= unique_labels(y_test, y_pred), yticklabels=unique_labels(y_test, y_pred))
plt.xlabel('True label')
plt.ylabel('Predicted label')
plt.title('Confusion Matrix')
plt.show()

Perceptron

In [85]:
num_transformer = SimpleImputer(strategy='median')  
cat_transformer = Pipeline(steps=[  
    ('imputer', SimpleImputer(strategy='most_frequent')),  
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  
])
preprocessor = ColumnTransformer(  
transformers=[  
    ('num', num_transformer, num_features),  
    ('cat', cat_transformer, cat_features)  
])

In [86]:
X = df.drop('Response', axis=1)  
y = df['Response']

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

In [90]:
X_train

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
172629,Male,55,1,8.0,0,1-2 Year,Yes,2630.0,124.0,201
88130,Female,63,1,28.0,0,1-2 Year,Yes,46987.0,26.0,243
286204,Female,38,1,8.0,1,1-2 Year,No,2630.0,7.0,130
319432,Male,41,1,28.0,0,> 2 Years,Yes,43999.0,124.0,208
52451,Male,67,1,28.0,1,1-2 Year,No,39608.0,52.0,282
...,...,...,...,...,...,...,...,...,...,...
282944,Female,28,1,18.0,1,< 1 Year,No,48483.0,152.0,89
357620,Female,48,1,28.0,1,1-2 Year,No,42327.0,124.0,185
332085,Female,55,1,7.0,0,> 2 Years,Yes,52000.0,156.0,130
188984,Male,27,1,30.0,0,< 1 Year,No,26356.0,154.0,58


In [91]:
#The sigmoid (non linear function) 
def sigmoid(z):
    ## compute the sigmoid
    return 1.0/(1+np.exp(-z))

In [92]:
#One hot function
def one_hot(y):
    n_classes = np.sort(np.unique(y))
    values = {n_class: i for i, n_class in enumerate(n_classes)}
    onehot = np.zeros((len(y), len(n_classes)))
    for idx, val in enumerate(y):
        onehot[ idx, values[val]] = 1.
    return onehot

In [93]:
#Cost function - logistic cost function
def cost_function(y_label, prediction):
    term1 = y_label * (np.log(prediction))
    term2 = (1. - y_label) * np.log(1. - prediction)
    return -np.sum(term1 + term2)/len(y_label)

In [94]:
#Forward propagation
def forward(X, w_h, b_h, w_out, b_out):
    z_h = np.dot(X, w_h) + b_h
    a_h = sigmoid(z_h)
    
    z_out = np.dot(a_h, w_out) + b_out
    a_out = sigmoid(z_out)
    
    return z_h, a_h, z_out, a_out

In [95]:
class MultiLayerPerceptron:
    def __init__(self, n_features, n_hidden, n_output):
        self.b_h = np.zeros(n_hidden)
        self.w_h = np.random.normal(loc=0.0, scale=0.1, size=(n_features, n_hidden))

        # weights for hidden -> output
        self.b_out = np.zeros(n_output)
        self.w_out = np.random.normal(loc=0.0, scale=0.1, size=(n_hidden, n_output))
        
    def forward(self, X):
        z_h = np.dot(X, self.w_h) + self.b_h
        a_h = sigmoid(z_h)
    
        z_out = np.dot(a_h, self.w_out) + self.b_out
        a_out = sigmoid(z_out)
        
        return z_h, a_h, z_out, a_out
    
   
    def backprop(self, X_batch, y_batch, z_h, a_h, z_out, a_out):
        delta_out = a_out - y_batch

        sigmoid_derivative_h = a_h * (1. - a_h)

        delta_h = (np.dot(delta_out, self.w_out.T) *
                   sigmoid_derivative_h)

        grad_w_h = np.dot(X_batch.T, delta_h)
        grad_b_h = np.sum(delta_h, axis=0)

        grad_w_out = np.dot(a_h.T, delta_out)
        grad_b_out = np.sum(delta_out, axis=0)

        return grad_w_h, grad_b_h, grad_w_out, grad_b_out
    
    def train(self, X_train, y_train, X_val, y_val, epochs, batch_size, learning_rate):
        epoch_loss_train = []
        epoch_loss_val = []
        batch_loss = []
        
        for idx_epoch in range(epochs):
            # iterate over minibatches
            indices = np.arange(X_train.shape[0])
            batch_num = 1
            random.shuffle(indices)
            for start_idx in range(0, indices.shape[0] - batch_size + 1, batch_size):
                batch_idx = indices[start_idx:start_idx + batch_size]

                X_batch = X_train[batch_idx]
                y_batch = y_train[batch_idx]
                
                # forward propagation
                z_h, a_h, z_out, a_out = self.forward(X_batch)
                
                batch_cost = cost_function(y_batch, a_out)
                batch_num += 1
                
                batch_loss.append(batch_cost)
                grad_w_h, grad_b_h, grad_w_out, grad_b_out = self.backprop(X_batch, y_batch, z_h, a_h, z_out, a_out)

                # Regularization and weight updates
                #delta_w_h = (grad_w_h + reg_rate*self.w_h)
                #delta_b_h = grad_b_h # bias is not regularized
                self.w_h -= learning_rate * grad_w_h
                self.b_h -= learning_rate * grad_b_h

                #delta_w_out = (grad_w_out + reg_rate*self.w_out)
                #delta_b_out = grad_b_out  # bias is not regularized
                self.w_out -= learning_rate * grad_w_out
                self.b_out -= learning_rate * grad_b_out
            
            _, _, _, a_out = self.forward(X_train)
            
            cost = cost_function(y_train, a_out)
            self.predict(X_val)
            
            _, _, _, a_out_val = self.forward(X_val)
            cost_val = cost_function(y_val, a_out_val)
            epoch_loss_train.append(cost)
            epoch_loss_val.append(cost_val)
            print(f'Epoch {idx_epoch}: train {cost} - eval {cost_val}')
        return epoch_loss_train, epoch_loss_val, batch_loss
    
    def predict(self, X):
        _, _, _, prediction = self.forward(X)
        return np.argmax(prediction, axis=1)
    
    def predict_proba(self, X):
        _, _, _, prediction = self.forward(X)
        return prediction


In [96]:
nn_code = MultiLayerPerceptron(X_train.shape[1], 30, n_output=10)

In [97]:
epoch_value, epoch_loss_val, batch_value = nn_code.train(X_train, y_train, X_val, y_val, epochs=150, batch_size=16, learning_rate=3*1e-4)

KeyError: "None of [Index([ 93093, 141391, 286577, 101209, 236621, 112983, 226863, 134220, 220318,\n       111839, 207476, 181139,  51254, 184393, 127394, 251095],\n      dtype='int64')] are in the [columns]"

Accuracy metrics for train, test and valuation

In [53]:
(np.argmax(y_train, axis=1) == np.argmax(nn_code.forward(X_train)[3], axis=1)).sum()/len(X_train)

ValueError: `axis` must be fewer than the number of dimensions (1)

In [ ]:
(np.argmax(y_test, axis=1) == np.argmax(nn_code.forward(X_test)[3], axis=1)).sum()/len(X_test)

In [ ]:
(np.argmax(y_val, axis=1) == np.argmax(nn_code.forward(X_val)[3], axis=1)).sum()/len(X_val)

In [ ]:
#Plot a graph having epoch loss for train and validation sets
x_epoch = np.linspace(0, len(epoch_value), len(epoch_value))
plt.plot(x_epoch, epoch_value, label='train epoch')
plt.plot(x_epoch, epoch_loss_val, label='validation epoch')
plt.legend()
plt.show()

In [ ]:
#Add to the plot a graph having epoch loss for batch loss
x_epoch = np.linspace(0, len(epoch_value), len(epoch_value))
batch_epoch = np.linspace(0, len(epoch_value), len(batch_value))
plt.plot(batch_epoch, batch_value, label='batch epoch')
plt.plot(x_epoch, epoch_value, label='train epoch')
plt.plot(x_epoch, epoch_loss_val, label='validation epoch')
plt.legend()
plt.show()